- author: Lee Meng
- date: 2019-03-25 09:00
- title: 讓 AI 寫點金庸：如何用 TensorFlow 2.0 及 TensorFlow.js 寫天龍八部
- slug: how-to-generate-interesting-text-with-tensorflow2-and-tensorflow-js
- tags: TensorFlow, TensorFlow.js, 自然語言處理
- description: TODO
- summary: TODO
- image: text-generation-cover.jpg
- image_credit_url: 
- status: draft

<link rel="stylesheet" href="{static}tfjs-apps/lstm-text-generation/index.css" />

!quote
- 木婉清轉頭向他，背脊向著南海鱷神，低聲道：「你是世上第一個見到我容貌的男子！」緩緩拉開了面幕。段譽登時全身一震，眼前所見，如新月清暉，如花樹堆雪，一張臉秀麗絕俗。
- 第四回：崖高人遠

<br/>

[《天龍八部》](https://bit.ly/2TUycBQ)一直是我最喜歡的[金庸著作](https://zh.wikipedia.org/wiki/%E9%87%91%E5%BA%B8%E4%BD%9C%E5%93%81)之一，最近重新翻閱，有很多新的感受。

閱讀到一半我突發奇想，決定用[深度學習](https://leemeng.tw/deep-learning-resources.html)來訓練一個能夠生成《天龍八部》的[循環神經網路](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E6%9C%89%E8%A8%98%E6%86%B6%E7%9A%84%E5%BE%AA%E7%92%B0%E7%A5%9E%E7%B6%93%E7%B6%B2%E8%B7%AF)。訓練結果還不完美，但我認為已經很有娛樂性質，且有時已經能夠產生令人驚嘆或是捧腹大笑的文章了。

因此我決定將模型上線，讓你也能看看這個 AI 實際上嗑了什麼藥。

!image
- lstm-text-generation/dali-old-castle.jpg
- 大理古城一隅，段譽出身之地

在 demo 之後，我將簡單地說明你可以如何使用 [TensorFlow 2.0 Alpha](https://www.tensorflow.org/alpha) 以及 [Tensorflow.js](https://www.tensorflow.org/js) 來實作這樣的 AI 應用。

希望你能從中得到些啟發，並在閱讀本文後實際動手做點自己的東西。

## Demo：生成全新的天龍八部橋段

本篇使用一個十分簡單的[長短期記憶 RNN](https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E8%A8%98%E6%86%B6%E5%8A%9B%E5%A5%BD%E7%9A%84-LSTM-%E7%B4%B0%E8%83%9E) 來生成文章。在多次「閱讀」天龍八部之後，這個模型可以在給定一段新文本的情況下，逐字產生類似風格的文章。

比方說給定書中的一個橋段：

```text
蕭峯吃了一驚，心想：「哥哥大喜之餘，說話有些忘形了，眼下亂成
```

模型順著上面的話，接著產生：

```text
一團，一大半也是個抵敵不住。

那女童見眾親兵到了幫外，只聽得遠處人影，都是迅速如飛，氣息直流。
```

這跟金庸的原作完全沒有關係，但感覺有模有樣。（至少我自己寫不出這樣的內容）

現在讓我們產生一些新的橋段吧！首先將已經訓練好的模型載入你的瀏覽器。依據不同網路速度，載入時間可能有所差異。

不過載入後只要不重新整理此頁面，你將可以不斷地使用此模型產生新的文章：

<section style="margin-bottom: 3rem">
    <button id="load-model" style="display:inline-block">載入模型</button>
    <div id="app-status" style="display:inline-block"></div>
</section>

另外你會發現有 2 個可供你調整的參數：

<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">生成長度（字單位）</span>
        <input id="generate-length" value="150"></input>
    </div>
    <div>
        <span class="input-title">生成溫度（隨機度）</span>
        <input id="temperature" value="0.6"></input>
    </div>
</section>

第一次可以直接使用預設值。現在點擊「生成文章」來產生全新的天龍八部橋段：

<section style="margin-bottom: 3rem">
    <div>
        <button id="generate-text" disabled="true">生成文章</button>
        <button id="initialize-seed" disabled="true">重置輸入</button>
    </div>
</section>
<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">起始句子：</span>
        <span id="text-generation-status" style="display: none"></span>
        <textarea id="seed-text" value="" rows="1" style="min-height: 6em">蕭峯吃了一驚，心想：「哥哥大喜之餘，說話有些忘形了，眼下亂成</textarea>
    </div>
</section>
<section style="margin-bottom: 3rem">
    <div>
        <span class="input-title">生成結果：</span>
        <textarea id="generated-text" readonly="true" value="" rows="10"></textarea>
    </div>
</section>

如何？希望模型產生的結果有令你會心一笑。它可快把我逗死了。

現在你可以做幾件事情來「玩弄」模型：
- 再次點擊「生成文章」來讓模型依據同樣輸入來產生新橋段
- 點擊「重置輸入」來隨機取得一個新的起始句子（或自己輸入）
- 增加模型生成的文章長度（耐心是美德）
- 調整「生成溫度」來改變文章的變化性

!image
- lstm-text-generation/chris-rhoads-254898-unsplash.jpg

生成溫度是一個介於 0 到 1 的值，而當溫度越高，模型產生出來的結果越隨機、越不可預測（也就越 ㄎㄧㄤ ）；而當你將溫度調低後，產生的結果會越像天龍八部原文。真實，但重複性也會提升。

找到一個最佳的溫度是個重要的事情。

如果你沒有打算深入探討技術細節，只需要記得在這篇文章裡頭的模型是一個以「字」為單位的語言模型（Character-based Language Model）即可：給定一連串已經出現過的字詞，模型會想辦法去預測出下一個可能出現的字。

!image
- lstm-text-generation/raychan-1229841-unsplash.jpg

值得注意的是，我們並不單純是拿出現機率最高的字出來當結果。每次預測我們都會得到一個所有字的出現機率分布，而在生成文本時，我們會對該機率分佈裡做隨機抽樣（Sample），讓模型每次都能生成既隨機又有點意思的字。

因此，每次產生的結果都是獨一無二的。如果你在生成的過程中得到什麼很有趣的虛擬橋段，都歡迎跟我分享！

## 所以這模型是怎麼訓練出來的

這模型嗑了什麼，也給我來一點




；生成溫度（Temperature）我們在後面還會有進一步解釋，但現在你只要知道它



在訓練過程中，模型會「閱讀」從《天龍八部》中擷取出來的大量句子，並試著從這些句子中預測出下一個可能出現的字。

## 章節內容


## 使用 TensorFlow 2.0 訓練模型

## 使用 Tensorflow.js 部署模型

## 結語

In [ ]:
#ignore
以下為 tfjs app 部署用的 js。記得每次 `yarn build` 以後都要使用最新的 js 

<script src="{static}tfjs-apps/lstm-text-generation/dist/lstm-text-generation.03657dc5.js"></script>